## Combined __parallel__ __masked__ and __taskloop__ Constructs

Just as the __for__ and __do__ constructs were combined with the __parallel__ construct for convenience, so too, the combined __parallel__ __masked__ __taskloop__ and  __parallel__ __masked__ __taskloop__ __simd__ constructs have been created for convenience when using the __taskloop__ construct.

In the following example the first __taskloop__ construct is enclosed by the usual __parallel__ and __masked__ constructs to form a team of threads, and a single task generator (primary thread) for the __taskloop__ construct.

The same OpenMP operations for the first taskloop are accomplished by the second taskloop with the __parallel__ __masked__ __taskloop__  combined construct.  The third taskloop uses the combined __parallel__ __masked__ __taskloop__ __simd__  construct to accomplish the same behavior as closely nested __parallel masked__, and __taskloop simd__ constructs.

As with any combined construct the clauses of the components may be used with appropriate restrictions. The combination of the __parallel__ __masked__ construct with the __taskloop__ or __taskloop__ __simd__ construct produces no additional  restrictions.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: parallel_masked_taskloop.1
* type: C
* version: omp_5.1
*/
#include <stdio.h>
#define N 100

int main()
{
   int i, a[N],b[N],c[N];

   for(int i=0; i<N; i++){ b[i]=i; c[i]=i; }  //init

   #pragma omp parallel
   #pragma omp masked
   #pragma omp taskloop                      // taskloop 1
   for(i=0;i<N;i++){ a[i] = b[i] + c[i]; }

   #pragma omp parallel masked taskloop      // taskloop 2
   for(i=0;i<N;i++){ b[i] = a[i] + c[i]; }

   #pragma omp parallel masked taskloop simd // taskloop 3
   for(i=0;i<N;i++){ c[i] = a[i] + b[i]; }

   printf(" %d %d\n",c[0],c[N-1]);  // 0 and 495
}

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name:       parallel_masked_taskloop.1
! type:       F-free
! version:    omp_5.1
program main

   integer, parameter :: N=100
   integer :: i, a(N),b(N),c(N)

   do i=1,N                            !! initialize
      b(i) = i
      c(i) = i
   enddo

   !$omp parallel
   !$omp masked
   !$omp taskloop                      !! taskloop 1
   do i=1,N
      a(i) = b(i) + c(i)
   enddo
   !$omp end taskloop
   !$omp end masked
   !$omp end parallel

   !$omp parallel masked taskloop      !! taskloop 2
   do i=1,N
      b(i) = a(i) + c(i)
   enddo
   !$omp end parallel masked taskloop

   !$omp parallel masked taskloop simd !! taskloop 3
   do i=1,N
      c(i) = a(i) + b(i)
   enddo
   !$omp end parallel masked taskloop simd

   print*,c(1),c(N)  !! 5 and 500

end program